In [4]:
from gurobipy import Model, quicksum, GRB
# from mypulp import Model, quicksum, GRB
from collections import OrderedDict, defaultdict
import networkx as nx

# データの中身確認

In [85]:
fname = "./data/ptask/data_10_51_111_66.dat"
f = open(fname, "r")
lines = f.readlines()
f.close()

In [86]:
n_jobs = int(lines[4].split()[2])
print("number of jobs=", n_jobs)
start, finish = OrderedDict(), OrderedDict()
for j in range(n_jobs):
    start[j], finish[j] = map(int, lines[5 + j].split())

number of jobs= 111


In [87]:
n_workers = int(lines[5 + n_jobs].split()[2])
print("number of workers=", n_workers)
Jobs = defaultdict(set)
for w in range(n_workers):
    line = lines[6 + n_jobs + w].split()
    for j in line[1:]:
        Jobs[w].add(int(j))

number of workers= 51


# 必要関数定義

In [88]:
def intersect(j, k):
    if start[j] > start[k]:
        j, k = k, j
    if finish[j] > start[k]:
        return True
    else:
        return False

In [89]:
G = nx.Graph()
for j in range(n_jobs):
    for k in range(j + 1, n_jobs):
        if intersect(j, k):
            G.add_edge(j, k)

In [90]:
Cliques = []
for c in nx.find_cliques(G):
    Cliques.append(set(c))
print("Number of cliques=", len(Cliques))

Number of cliques= 32


# 最適化モデリング

## gurobipyを使う場合

In [84]:
model = Model()
x, y = {}, {}
for w in range(n_workers):
    y[w] = model.addVar(vtype="B", name=f"y[{w}]")
    for j in Jobs[w]:
        x[j, w] = model.addVar(vtype="B", name=f"x[{j},{w}]")
model.update()
for j in range(n_jobs):
    model.addConstr(
        quicksum(x[j, w] for w in range(n_workers) if (j, w) in x) == 1,
        name=f"job_assign[{j}]",
    )
for w in range(n_workers):
    for j, c in enumerate(Cliques):
        model.addConstr(
            quicksum(x[j, w] for j in Jobs[w].intersection(c)) <= y[w],
            name=f"connection[{j},{w}]",
        )
model.setObjective(quicksum(y[w] for w in range(n_workers)), GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15 rows, 34 columns and 59 nonzeros
Model fingerprint: 0xb6a2f7ea
Variable types: 0 continuous, 34 integer (34 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 18 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


## pulpを使う場合

In [91]:
import pulp

# 問題定義
problem = pulp.LpProblem("schedule", sense=pulp.LpMinimize)

# 変数定義
x_jw = [[pulp.LpVariable(f"x_{j},{w}", cat=pulp.LpBinary) for w in range(n_workers)] for j in range(n_jobs)]
y_w = [ pulp.LpVariable( f"y_{w}", cat=pulp.LpBinary ) for w in range(n_workers) ]

# 目的関数定義
problem += pulp.lpSum(y_w)


# 制約関数定義
for j in range(n_jobs):
    problem += pulp.lpSum(x_jw[j]) == 1

for w in range(n_workers):
    for c in Cliques:
        problem += pulp.lpSum([x_jw[j][w] for j in range(n_jobs) if j in Jobs[w].intersection(c)]) <= y_w[w]

# (6) 解く
result = problem.solve()

# 結果を出力
print('result: {}'.format(pulp.LpStatus[result]))

print('objective value: {}'.format(pulp.value(problem.objective)))
print('solution')

for j in range(n_jobs):
    for w in range(n_workers):
        print(f"x_{j},{w}={pulp.value(x_jw[j][w])}")

for w in range(len(y_w)):
    print(f"y_{w}={pulp.value(y_w[w])}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yamauchito_satoshi/Documents/code/ShiftOptimization/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/39/_5nhjpmd3jjc437p6ttrf5vm0000gn/T/6c3151363c1a4561b0dc24f05b11fdcf-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/39/_5nhjpmd3jjc437p6ttrf5vm0000gn/T/6c3151363c1a4561b0dc24f05b11fdcf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1748 COLUMNS
At line 63244 RHS
At line 64988 BOUNDS
At line 70701 ENDATA
Problem MODEL has 1743 rows, 5712 columns and 50020 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 397 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 223 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 115 strengthened rows,

In [7]:
import pandas as pd

df = pd.DataFrame([1, 2, 3])
sample_csv = df.to_csv(index=False)

In [21]:
df_request_table = pd.read_csv("sample.csv", index_col=0, header=0)

In [22]:
df_request_table

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
A,3,1,1,1,3,1,3,2,1,1,...,2,2,2,2,3,1,3,2,3,3
B,3,2,1,1,2,3,2,1,2,2,...,1,3,2,3,1,3,1,1,1,3
C,3,2,2,1,2,2,3,2,3,1,...,2,1,3,1,2,2,3,3,1,1
D,3,3,2,1,2,2,1,3,3,2,...,3,3,2,3,3,3,2,3,2,2
E,1,3,1,2,3,1,3,3,3,2,...,1,3,3,2,1,1,1,1,2,2
F,3,3,3,1,3,1,1,2,2,3,...,1,3,2,1,3,3,1,1,3,1
G,2,3,1,2,3,3,2,2,3,3,...,3,1,1,3,3,1,1,2,2,1
